## Libraries

In [1]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
from rioxarray.merge import merge_arrays

C:\Users\enhi\AppData\Local\Temp\ipykernel_9092\4077202506.py:4: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


## The aim of this Notebook is to mosaic 4 tiles and clip them to AOI using long time series data

In [3]:
main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly'
# Output directory
Clipped_path = os.path.join(main_dir, 'Clipped_FAPAR')
# Directories for each tile
Boundary = os.path.join(main_dir, 'Boundary')
Tile_1 = os.path.join(main_dir, 'tile_1')
Tile_2 = os.path.join(main_dir, 'tile_2')
Tile_3 = os.path.join(main_dir, 'tile_3')
Tile_4 = os.path.join(main_dir, 'tile_4')
# Lists of images' names for each tile
Images_in_tile1 = sorted(os.listdir(Tile_1))
Images_in_tile2 = sorted(os.listdir(Tile_2))
Images_in_tile3 = sorted(os.listdir(Tile_3))
Images_in_tile4 = sorted(os.listdir(Tile_4))
# Directory to an example of Copernicus FAPAR to take the shape and crs fro it
Copernicus_ex =rxr.open_rasterio(r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\FAPAR anomaly\Monthly_FAPAR_Glass\April\2000April.tif', masked =True)

In [3]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shape file desired to mask a grid.
    Args:
        filepath: Path to *.shp file
    """
    shpfile = gpd.read_file(filepath)
    print("""Shapefile loaded. To prepare for masking, run the function
        `select_shape`.""")
    return shpfile

#Create the mask
def select_shape(shpfile):

    """Select the submask of interest from the shapefile.
    Args:
        shpfile: (*.shp) loaded through `load_shape_file`
        category: (str) header of shape file from which to filter shape.
            (Run print(shpfile) to see options)
        name: (str) name of shape relative to category.
           Returns:
        shapely polygon
    """

    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']

    # Extract the rows that have 'ZAF', 'LSO', or 'SWZ' in the 'SOV_A3' column
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]

    # Combine the selected polygons into a single polygon
    unioned_polygon = selected_rows.geometry.unary_union

    # Convert the unioned polygon to a geopandas dataframe with a single row
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    
    print("""Mask created.""")

    return mask_polygon
shpfile = load_shape_file(Boundary + '\\'+ 'CNTR_RG_01M_2020_4326.shp')
merged = select_shape(shpfile)

Shapefile loaded. To prepare for masking, run the function
        `select_shape`.
Mask created.


In [4]:
# crs of FAPAR copernicus
Copernicus_ex_crs = Copernicus_ex.rio.crs
# Define a crs to the AOI 
merged.crs =Copernicus_ex_crs

In [5]:
# looping based on the number of images
for i in range (0 , len(Images_in_tile4)):
    # Open corresponding images from each tile
    tile1 = rxr.open_rasterio(Tile_1 + '\\'+ Images_in_tile1[i], masked=True)
    tile2 = rxr.open_rasterio(Tile_2 + '\\'+ Images_in_tile2[i], masked=True)
    tile3 = rxr.open_rasterio(Tile_3 + '\\'+ Images_in_tile3[i], masked=True)
    tile4 = rxr.open_rasterio(Tile_4 + '\\'+ Images_in_tile4[i], masked=True)
    # Mosiac of the corresponding images of each tile by using only NDVI bands
    merged_raster = merge_arrays(dataarrays = [tile1, tile2,
                                               tile3,tile4])
    # Clipping the mosiac to the AOI
    clipped = merged_raster.rio.clip(merged.geometry.apply(mapping),
                                     crs=merged.crs,
                                     all_touched=True,
                                     from_disk=True).squeeze()
    # New directory includes output clipped images of each year
    # Note that [9:13] is used as to extract the year from the name of images
    year = Clipped_path + '\\'+ Images_in_tile1[i][16:20]
    check_folder_year = os.path.isdir(year)
    if not check_folder_year:
        os.makedirs(year)
    # Export clipped images
    clipped.rio.to_raster(year + '\\'+ Images_in_tile1[i]+ '.tif')

C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\envs\rasterio\Lib\site-packages\rioxarray\raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
C:\Users\enhi\Anaconda3\